In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import matplotlib.mlab as mlab
from scipy.optimize import leastsq
import datetime
%matplotlib inline

/Users/gabrielfior/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
#Pre-process file
#filename = 'ArgonValidation_nt_B4_g.csv'
filename = '10cm_1e6_400GeV_Ar_cut1_5mm_nt_B4_g.csv'
t0 = datetime.datetime.now()
df = pd.read_csv('/Volumes/UNTITLED/'+filename,
                 skiprows=19,header=None,names=['trackId','parentId','volumeName','particleName','stepNumber',
                                               'posX','posY','posZ','perp','stepLength','kineeticEnergyDiff',
                                               'edepStep','kineticEnergyPostStep','processName','fCluster'
                                                ])
count1=0
list1=[]
print 'reading: elapsed '+str(datetime.datetime.now()-t0)
t0 = datetime.datetime.now()
for i in range(df.shape[0]):
    
    #if i%1000==0:
    #    print 'i= ' + str(i)
    if df.iloc[i]['processName']=='event':
        count1 +=1
    else:
        list1.append(count1)    

print 'processing: elapsed '+str(datetime.datetime.now()-t0)

In [ ]:
#delete unwanted columns


In [ ]:
df_events = df[df.processName=='event']
df_steps = df[df.processName!='event']

df_steps.insert(len(df_steps.columns),'particleId',np.array(list1))
#print df_steps.particleName.unique()

In [ ]:
"""
NOT NECESSARY BECAUSE OF PRE-PROCESSING BEFORE
#reading data frame
#filename = 'testem8_nt_B4_g.csv'
#filename = 'B4_nt_B4_g_origin_processed.csv'
filename = 'ArgonValidation_nt_B4_g.csv'
#filename = 'argon_400geV_ntp.csv'
#df = pd.read_csv('/Users/gabrielfior/OneDrive/Master Thesis/Share_Ubuntu/'+filename,
#                 skiprows=7,header=None,names=['energyDeposited_ev','fClusters','whatever'])
df = pd.read_csv('/Volumes/UNTITLED/'+filename,
                 skiprows=19,header=None,names=['trackId','parentId','volumeName','particleName','stepNumber',
                                               'posX','posY','posZ','perp','stepLength','kineeticEnergyDiff',
                                               'edepStep','kineticEnergyPostStep','processName','fCluster',
                                                'particleId'])
sf = graphlab.SFrame.read_csv('/Volumes/UNTITLED/'+filename,)    
#argon_400geV_ntp.csv
"""

In [ ]:
#df.head()

In [ ]:
#df.head(40)

#df_events=df[df.processName=='event']
df_steps_p = df_steps[(df_steps.particleName=='proton')]
df_steps_e = df_steps[(df_steps.particleName=='e-')]

#df_steps_p = df[(df.particleName=='proton')]
#df_steps_e = df[(df.particleName=='e-')]


#del (df)
#print df.shape
#np.mean(df.energyDeposited_ev)
#df.energyDeposited_ev.unique()

In [ ]:
#Group by stepNumber
agg1 = {'edepStep':['sum'], 'stepNumber': ['max'], 'kineticEnergyPostStep':['min','max'],'fCluster':['sum']}
df_grouped_p = df_steps_p.groupby('particleId',as_index=False,group_keys=False).agg(agg1)

#idx = df.groupby('word')['count'].idxmax()
#print df_steps_p.stepNumber

# Mean energy deposit by protons

In [ ]:
print (df_grouped_p['edepStep'].mean())
df_grouped_p.hist(['edepStep'],bins=np.linspace(0,7e3,100))
plt.show()
#plt.hist(df_grouped_p,bins=np.linspace(0,7000,100))
#plt.title('Mean energy deposited by protons: ' + str(df_grouped_p.mean()))
#plt.show()

In [ ]:
# Mean Clusters generated by proton

print (df_grouped_p['fCluster'].mean())
df_grouped_p.hist(['fCluster'],bins=np.linspace(0,50,100))
plt.show()


# Exploring df_events 

In [ ]:
df_events.head()

In [ ]:
plt.subplot(121)
plt.hist(df_events.kineticEnergyPostStep,bins=np.linspace(0,7000,100))
plt.title(str(df_events.kineticEnergyPostStep.mean()))
plt.subplot(122)
plt.hist(df_events.fCluster,bins=np.linspace(0,600,100))
plt.title(str(df_events.fCluster.mean()))
plt.show()

# Fit histograms and get mean, rms

In [ ]:
# Generate some data for this demonstration.
#data = norm.rvs(10.0, 2.5, size=500)
def return_fitted_gaussian_parameters(data1,bins1):
    #data = df.fClusters
    data = data1
    # Fit a normal distribution to the data:
    mu, std = norm.fit(data)

    # Plot the histogram.
    nbins = len(bins1)
    result = plt.hist(data, bins=bins1, normed=False, color='g')
    dx = result[1][1] - result[1][0]
    scale = len(data)*dx
    
    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, nbins)
    p = norm.pdf(x, mu, std)*scale
    plt.plot(x, p, 'k', linewidth=2)
    title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
    plt.title(title)
    plt.show()
        
    return [mu,std]

In [ ]:
a,b=return_fitted_gaussian_parameters(df_events.fCluster,np.linspace(0,600,100))
print a,b

In [ ]:
print 'mean value: '+str(df_events.kineticEnergyPostStep.mean())+' kEV'
return_fitted_gaussian_parameters(df_events.kineticEnergyPostStep,np.linspace(0,8000,100))

# Investigate df_steps

In [ ]:
#kinetic energy diff sum = edepstep sum + final energy electrons
total_energy_deposit = df_steps_e.edepStep.sum() + df_steps_p.edepStep.sum()

total_kin_diff = df_steps_p.kineeticEnergyDiff.sum()
print -total_kin_diff,total_energy_deposit
print 'diference in energy: '+str(-total_kin_diff+total_energy_deposit)

In [ ]:
#Get minimum kinectic energy of e-
agg1 = {'kineticEnergyPostStep':['min']}
df_grouped_e = df_steps_e.groupby('particleId',as_index=False,group_keys=False).agg(agg1)
#print df_grouped_e.head()
#print df_grouped_e['kineticEnergyPostStep'].sum()
print df_steps_e.head()

In [ ]:
#print df_steps[df_steps.particleId==0]['kineeticEnergyDiff'].sum()
print df_steps[df_steps.particleId==0]['edepStep'].sum()
#print df_steps_p[df_steps_p.particleId==0]['edepStep'].sum()
print df_steps_p[df_steps_p.particleId==0]['kineeticEnergyDiff'].sum()

In [ ]:
#df_steps_e[df_steps_e.particleId==0]#227.165
print df_steps_p[df_steps_p.particleId==0]['edepStep'].sum()
print df_steps_p[df_steps_p.particleId==0]['kineeticEnergyDiff'].sum()

In [ ]:
2281.717 - 2054.55253